In [1]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import Reader
from surprise import Trainset

from collections import defaultdict
from operator import itemgetter
import heapq

import os
import csv

import pandas as pd

In [68]:
os.getcwd()

'/Users/rukunzhang/code/saratsiri/anime-rec/notebooks'

In [70]:
os.path.dirname(os.getcwd())

'/Users/rukunzhang/code/saratsiri/anime-rec'

In [72]:
file_path=f"{os.path.dirname(os.getcwd())}/processed_data/similarity_matrix_full.npy"
file_path

'/Users/rukunzhang/code/saratsiri/anime-rec/processed_data/similarity_matrix_full.npy'

In [73]:
file_path=f"{os.path.dirname(os.getcwd())}/processed_data/similarity_matrix_full.npy"
# load array
similarity_matrix = load(file_path)

In [74]:
similarity_matrix

array([[1.        , 0.9613837 , 0.97060007, ..., 0.        , 0.        ,
        0.        ],
       [0.9613837 , 1.        , 0.97409777, ..., 0.        , 0.        ,
        0.        ],
       [0.97060007, 0.97409777, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [66]:
file_path

'../processed_data/similarity_matrix_full.npy'

In [46]:
# load numpy array from npy file
from numpy import load

# load array
similarity_matrix = load('../processed_data/similarity_matrix_full.npy')


In [6]:
AnimesDF = pd.read_csv('../raw_data/anime_cleaned.csv')
ScoresDF = pd.read_csv('../raw_data/animelists_cleaned.csv')

In [63]:
animeID_to_name = AnimesDF.set_index('anime_id')['title'].to_dict()


In [48]:
ScoresDF_selected= ScoresDF[ScoresDF["my_score"]>0][["username","anime_id","my_score","my_last_updated"]]

In [8]:
#small_fraction = 0.0001
#ScoresDF_selected_small = ScoresDF_selected.sample(int(len(ScoresDF_selected)*small_fraction))
#ScoresDF_selected_small.shape

(1917, 4)

In [49]:
ScoresDF_selected.head()

,username,anime_id,my_score,my_last_updated
0,karthiga,21,9,2013-03-03 10:52:53
1,karthiga,59,7,2013-03-10 13:54:51
2,karthiga,74,7,2013-04-27 16:43:35
3,karthiga,120,7,2013-03-03 10:53:57
4,karthiga,178,7,2013-03-27 15:59:13


In [50]:
reader = Reader(rating_scale=(0, 10))
scoredata = Dataset.load_from_df(ScoresDF_selected[['username', 'anime_id', 'my_score']], reader)
trainset = scoredata.build_full_trainset()

In [58]:
# Pick a random user name.
# Play around and see how the final recommendations change
# depending on the user! 1-610
test_subject = 'roxamrit'

# Get the top K items user rated
k = 20

In [59]:
# When using Surprise, there are RAW and INNER IDs.
# Raw IDs are the IDs, strings or numbers, you use when
# creating the trainset. The raw ID will be converted to
# an unique integer Surprise can more easily manipulate
# for computations.
#
# So in order to find an user inside the trainset, you
# need to convert their RAW ID to the INNER Id. Read
# here for more info https://surprise.readthedocs.io/en/stable/FAQ.html#what-are-raw-and-inner-ids
test_subject_iid = trainset.to_inner_uid(test_subject)

# Get the top K items we rated
test_subject_ratings = trainset.ur[test_subject_iid]
k_neighbors = heapq.nlargest(k, test_subject_ratings, key=lambda t: t[1])



In [60]:
# Default dict is basically a standard dictionary,
# the difference beeing that it doesn't throw an error
# when trying to access a key which does not exist,
# instead a new entry, with that key, is created.
candidates = defaultdict(float)

for itemID, rating in k_neighbors:
    try:
      similaritities = similarity_matrix[itemID]
      for innerID, score in enumerate(similaritities):
          candidates[innerID] += score * (rating / 5.0)
    except:
      continue
  


In [61]:
# Utility we'll use later.
def getAnimeName(animeID):
  if int(animeID) in animeID_to_name:
    return animeID_to_name[int(animeID)]
  else:
      return ""

In [62]:
# Build a dictionary of movies the user has watched
watched = {}
for itemID, rating in trainset.ur[test_subject_iid]:
  watched[itemID] = 1

# Add items to list of user's recommendations
# If they are similar to their favorite movies,
# AND have not already been watched.
recommendations = []

position = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
  if not itemID in watched:
    recommendations.append(getAnimeName(trainset.to_raw_iid(itemID)))
    position += 1
    if (position > 10): break # We only want top 10

for rec in recommendations:
  print("Anime: ", rec)

Anime:  Pipi Tobenai Hotaru
Anime:  Go Go Toraemon
Anime:  Shima no Kuregata no Hanashi
Anime:  Peace Shitenchou
Anime:  Qin Shi Ming Yue: Tian Xing Jiu Ge
Anime:  Qin Shi Ming Yue: Yi Zhe Ru Shi
Anime:  Makiba no Shoujo Katri Specials
Anime:  Komaneko: Hajime no Ippo
Anime:  Town Workers
Anime:  Taisetsu na Kazoku wo Tsutsumu, Sono Te.
Anime:  Kinyoubi no Yakusoku


In [24]:
karthiga_list=ScoresDF[ScoresDF["username"]=="karthiga"]

In [25]:
karthiga_list

,username,anime_id,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,karthiga,21,586,0000-00-00,0000-00-00,9,1,NaN,0,2013-03-03 10:52:53,NaN
1,karthiga,59,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:54:51,NaN
2,karthiga,74,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:43:35,NaN
3,karthiga,120,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-03 10:53:57,NaN
4,karthiga,178,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN
5,karthiga,210,161,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:57:06,NaN
6,karthiga,232,70,0000-00-00,0000-00-00,6,2,NaN,0,2013-03-09 17:24:42,NaN
7,karthiga,233,78,0000-00-00,0000-00-00,6,2,NaN,0,2013-03-10 05:29:44,NaN
8,karthiga,249,167,0000-00-00,0000-00-00,8,2,NaN,0,2013-03-19 16:04:46,NaN
9,karthiga,269,366,0000-00-00,0000-00-00,10,2,NaN,0,2013-03-03 09:39:23,NaN


In [28]:
karthiga_sample1=AnimesDF.merge(karthiga_list,on="anime_id")

In [31]:
karthiga_sample1

,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,status,...,username,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,Finished Airing,...,karthiga,12,0000-00-00,0000-00-00,8,2,NaN,0,2013-05-05 14:24:59,NaN
1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,karthiga,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:32:21,NaN
2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,Finished Airing,...,karthiga,51,0000-00-00,0000-00-00,7,2,NaN,0,2013-05-05 14:23:55,NaN
3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,Finished Airing,...,karthiga,38,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:23:53,NaN
4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,Finished Airing,...,karthiga,25,0000-00-00,0000-00-00,8,2,NaN,0,2013-04-27 16:12:43,NaN
5,6586,Yume-iro Pâtissière,NaN,夢色パティシエール,"Yumeiro Patissiere, YumePati, Dream-Colored Pa...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,50,Finished Airing,...,karthiga,50,0000-00-00,0000-00-00,6,2,NaN,0,2013-03-10 14:18:30,NaN
6,178,Ultra Maniac,Ultramaniac - Magical Girl,ウルトラマニアック,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,karthiga,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN
7,2787,Shakugan no Shana II (Second),Shakugan no Shana: Season II,灼眼のシャナII –Second–,Shakugan no Shana 2,https://myanimelist.cdn-dena.com/images/anime/...,TV,Light novel,24,Finished Airing,...,karthiga,24,0000-00-00,0000-00-00,8,2,NaN,0,2013-04-27 16:41:10,NaN
8,4477,Nodame Cantabile: Paris-hen,NaN,のだめカンタービレ 巴里編,"Nodame Cantabile Paris Chapter, Nodame Cantabi...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,11,Finished Airing,...,karthiga,11,0000-00-00,0000-00-00,8,2,NaN,0,2013-04-27 16:21:21,NaN
9,853,Ouran Koukou Host Club,Ouran High School Host Club,桜蘭高校ホスト部,"Ohran Koko Host Club, Ouran Koukou Hosutobu, O...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,Finished Airing,...,karthiga,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:57:27,NaN


In [32]:
pd.read_csv("../processed_data/karthiga_sample1.csv")

,Unnamed: 0,anime_id,title,title_english,title_japanese,title_synonyms,image_url,type,source,episodes,...,username,my_watched_episodes,my_start_date,my_finish_date,my_score,my_status,my_rewatching,my_rewatching_ep,my_last_updated,my_tags
0,0,11013,Inu x Boku SS,Inu X Boku Secret Service,妖狐×僕SS,Youko x Boku SS,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,12,...,karthiga,12,0000-00-00,0000-00-00,8,2,NaN,0,2013-05-05 14:24:59,NaN
1,1,2104,Seto no Hanayome,My Bride is a Mermaid,瀬戸の花嫁,The Inland Sea Bride,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,...,karthiga,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:32:21,NaN
2,2,5262,Shugo Chara!! Doki,Shugo Chara!! Doki,しゅごキャラ！！どきっ,"Shugo Chara Ninenme, Shugo Chara! Second Year",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,51,...,karthiga,51,0000-00-00,0000-00-00,7,2,NaN,0,2013-05-05 14:23:55,NaN
3,3,721,Princess Tutu,Princess Tutu,プリンセスチュチュ,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Original,38,...,karthiga,38,0000-00-00,0000-00-00,7,2,NaN,0,2013-04-27 16:23:53,NaN
4,4,12365,Bakuman. 3rd Season,Bakuman.,バクマン。,Bakuman Season 3,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,25,...,karthiga,25,0000-00-00,0000-00-00,8,2,NaN,0,2013-04-27 16:12:43,NaN
5,5,6586,Yume-iro Pâtissière,NaN,夢色パティシエール,"Yumeiro Patissiere, YumePati, Dream-Colored Pa...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,50,...,karthiga,50,0000-00-00,0000-00-00,6,2,NaN,0,2013-03-10 14:18:30,NaN
6,6,178,Ultra Maniac,Ultramaniac - Magical Girl,ウルトラマニアック,NaN,https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,...,karthiga,26,0000-00-00,0000-00-00,7,2,0.0,0,2013-03-27 15:59:13,NaN
7,7,2787,Shakugan no Shana II (Second),Shakugan no Shana: Season II,灼眼のシャナII –Second–,Shakugan no Shana 2,https://myanimelist.cdn-dena.com/images/anime/...,TV,Light novel,24,...,karthiga,24,0000-00-00,0000-00-00,8,2,NaN,0,2013-04-27 16:41:10,NaN
8,8,4477,Nodame Cantabile: Paris-hen,NaN,のだめカンタービレ 巴里編,"Nodame Cantabile Paris Chapter, Nodame Cantabi...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,11,...,karthiga,11,0000-00-00,0000-00-00,8,2,NaN,0,2013-04-27 16:21:21,NaN
9,9,853,Ouran Koukou Host Club,Ouran High School Host Club,桜蘭高校ホスト部,"Ohran Koko Host Club, Ouran Koukou Hosutobu, O...",https://myanimelist.cdn-dena.com/images/anime/...,TV,Manga,26,...,karthiga,26,0000-00-00,0000-00-00,7,2,NaN,0,2013-03-10 13:57:27,NaN
